In [ ]:
import json

import bottle
import dbpediaknowledge
import solrindexer as indexer

# bottleのフレームワークはルーティングを1ファイル内ですべて完結できる
# 下記@bottle.route('/')内はホームルートで有ることを表す
@bottle.route('/')
def index_html():
    # ルート（http://localhost:8702）にアクセスした時に表示数rファイルをsearch.htmlと指定している。
    # rootにはフォルダまでを記述、ファイル名は第1引数に記述
    return bottle.static_file('search.html', root='../data/static')


@bottle.route('/file/<filename:path>')
def static(filename):
    # file(http://localhost:8702/file/)にアクセスした時に、data/staticフォルダの下のファイルを返す用にしている。
    return bottle.static_file(filename, root='../data/static')

# 下記はhttp://localhost:8702/getにアクセスした時に実行される
@bottle.get('/get')
def get():
    # リクエストの中からkeywordsと、nameという変数を受け取る
    name = bottle.request.params.name
    # keywordsはsplit()で空白文字で区切ってリストにする
    keywords = bottle.request.params.keywords.split()
    # keywordsの同義語を検索してkeywords_expandedに収納する
    keywords_expanded = [[keyword] + [synonym['term']
                                      for synonym in dbpediaknowledge.get_synonyms(keyword)]
                                         for keyword in keywords]
    # inputのkeywordsが空を想定する
    if keywords_expanded != []:
        fl_keyword_pairs = [(name + '_txt_ja', keywords_expanded)]
    else:
        fl_keyword_pairs = [('name_s', [[name]])]
        
    # search関数を呼びresultsのリストに代入する
    # results = indexer.search(keywords=keywords_expanded)
    # search_annotaion関数を呼びresultsのリストに代入する
    results = indexer.search_annotation(fl_keyword_pairs)
    # results変数をのjsonデータを返す
    return json.dumps(results, ensure_ascii=False)

# このファイルがモジュールとして実行された場合にファイル名が__namae__に入る。モジュールではなくメインのプログラムとして実行された場合__main__が__namae__に入りtrueとなる
if __name__ == '__main__':
# ポート番号を決めhttp://localhost:8702でアクセスできるようにしている
    bottle.run(host='0.0.0.0', port='8702')

Bottle v0.12.17 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:8702/
Hit Ctrl-C to quit.

127.0.0.1 - - [12/Nov/2019 02:58:40] "GET / HTTP/1.1" 304 0
127.0.0.1 - - [12/Nov/2019 02:58:40] "GET /file/search.js HTTP/1.1" 304 0
127.0.0.1 - - [12/Nov/2019 02:58:41] "GET /get?name=affiliation&keywords=%E3%82%A4%E3%83%B3%E3%83%89 HTTP/1.1" 200 4353
127.0.0.1 - - [12/Nov/2019 03:00:17] "GET / HTTP/1.1" 304 0
127.0.0.1 - - [12/Nov/2019 03:00:17] "GET /file/search.js HTTP/1.1" 200 1437
127.0.0.1 - - [12/Nov/2019 03:00:25] "GET /get?name=affiliation&keywords=%E3%82%A4%E3%83%B3%E3%83%89 HTTP/1.1" 200 4352
127.0.0.1 - - [12/Nov/2019 03:00:28] "GET / HTTP/1.1" 304 0
127.0.0.1 - - [12/Nov/2019 03:00:28] "GET /file/search.js HTTP/1.1" 304 0
127.0.0.1 - - [12/Nov/2019 03:00:32] "GET /get?name=affiliation&keywords=%E3%82%A4%E3%83%B3%E3%83%89 HTTP/1.1" 200 4352
127.0.0.1 - - [12/Nov/2019 03:03:38] "GET / HTTP/1.1" 200 1457
127.0.0.1 - - [12/Nov/2019 03:03:38] "GET /file/search.